In [1]:
from plb.engine.taichi_env import TaichiEnv
import taichi as ti
import numpy as np
from utils import *

from plb.config import load
cfg = load(f"envs/BendTube.yml") # you can find most default config is at plb/config/default_config.py



[Taichi] mode=release
[Taichi] preparing sandbox at /tmp/taichi-xsxklhpy
[Taichi] version 0.7.26, llvm 10.0.0, commit e37bdb5e, linux, python 3.7.11
[I 03/12/22 03:00:42.432 771207] [shell.py:_shell_pop_print@35] Graphical python shell detected, using wrapped sys.stdout
[Taichi] Starting on arch=cuda


In [2]:
cfg

CfgNode({'SIMULATOR': CfgNode({'dim': 3, 'quality': 1, 'yield_stress': 50.0, 'dtype': 'float64', 'max_steps': 1024, 'n_particles': 5000, 'E': 5000.0, 'nu': 0.2, 'ground_friction': 200.0, 'gravity': (0, -1, 0)}), 'PRIMITIVES': [{'shape': 'Box', 'size': '(0.03, 0.08, 0.03)', 'init_pos': '(0.2, 0.2, 0.2)', 'init_rot': '(0., 0., 0., 1.)', 'lower_bound': '(0., 0., 0.)', 'color': '(0.8, 0.8, 0.8)', 'friction': 0.0, 'action': {'dim': 3, 'scale': '(0.01, 0.01, 0.01)'}}, {'shape': 'Box', 'size': '(0.03, 0.08, 0.03)', 'init_pos': '(0.2, 0.2, 0.2)', 'init_rot': '(0., 0., 0., 1.)', 'lower_bound': '(0., 0., 0.)', 'color': '(0.8, 0.8, 0.8)', 'friction': 0.0, 'action': {'dim': 3, 'scale': '(0.01, 0.01, 0.01)'}}, {'shape': 'Box', 'size': '(0.03, 0.08, 0.03)', 'init_pos': '(0.2, 0.2, 0.2)', 'init_rot': '(0., 0., 0., 1.)', 'lower_bound': '(0., 0., 0.)', 'color': '(0.8, 0.8, 0.8)', 'friction': 0.0, 'action': {'dim': 3, 'scale': '(0.01, 0.01, 0.01)'}}], 'SHAPES': [{'shape': 'box', 'n_particles': 5000, 'wi

In [3]:
cfg['RENDERER']['sdf_threshold'] = 0.20720000000000002*2

In [4]:
env = TaichiEnv(cfg, nn=False, loss=False)

Building primitive
action:
  dim: 3
  scale: (0.01, 0.01, 0.01)
color: (0.8, 0.8, 0.8)
friction: 0.0
init_pos: (0.2, 0.2, 0.2)
init_rot: (0.0, 0.0, 0.0, 1.0)
lower_bound: (0.0, 0.0, 0.0)
shape: Box
size: (0.03, 0.08, 0.03)
upper_bound: (1.0, 1.0, 1.0)
variations: None
Building primitive
action:
  dim: 3
  scale: (0.01, 0.01, 0.01)
color: (0.8, 0.8, 0.8)
friction: 0.0
init_pos: (0.2, 0.2, 0.2)
init_rot: (0.0, 0.0, 0.0, 1.0)
lower_bound: (0.0, 0.0, 0.0)
shape: Box
size: (0.03, 0.08, 0.03)
upper_bound: (1.0, 1.0, 1.0)
variations: None
Building primitive
action:
  dim: 3
  scale: (0.01, 0.01, 0.01)
color: (0.8, 0.8, 0.8)
friction: 0.0
init_pos: (0.2, 0.2, 0.2)
init_rot: (0.0, 0.0, 0.0, 1.0)
lower_bound: (0.0, 0.0, 0.0)
shape: Box
size: (0.03, 0.08, 0.03)
upper_bound: (1.0, 1.0, 1.0)
variations: None
{'n_particles': 5000, 'width': (0.6, 0.1, 0.1), 'init_pos': (0.5, 0.05, 0.5), 'color': 13158400}
Initialize Renderer
bake_size: 6  
camera_pos: (0.5, 2.0, 1.2)  
camera_rot: (1.3, 0.0)  
dx: 0.

In [5]:
def animate(imgs, filename='animation.webm', _return=True, fps=60):
    print(f'animating {filename}')
    from moviepy.editor import ImageSequenceClip
    imgs = ImageSequenceClip(imgs, fps=fps)
    imgs.write_videofile(filename, fps=fps)
    if _return:
        from IPython.display import Video
        return Video(filename, embed=True)

In [6]:
def set_parameters(env: TaichiEnv, yield_stress, E, nu):
    env.simulator.yield_stress.fill(yield_stress)
    _mu, _lam = E / (2 * (1 + nu)), E * nu / ((1 + nu) * (1 - 2 * nu))  # Lame parameters
    env.simulator.mu.fill(_mu)
    env.simulator.lam.fill(_lam)

In [7]:
dt = 1. / 60.

grip_time = 1
time_step = 40
dim_position = 4
dim_velocity = 3
dim_shape_state = 14
rest_gripper_dis = 1.8


In [8]:
env.initialize()

set_parameters(env, 200, 5000, 0.2)

state = env.get_state()


In [18]:
# test out parameters
params = [
    [5, 100, 0],
    [5, 100, 0.45],
    [5, 3000, 0],
    [5, 3000, 0.45],
    [200, 100, 0],
    [200, 100, 0.45],
    [200, 3000, 0],
    [200, 3000, 0.45],
]

for param_i in range(len(params)):
    
    env.initialize()

    set_parameters(env, params[param_i][0], params[param_i][1], params[param_i][2])

    state = env.get_state()


    # gripper_config = sample_gripper_config("BendTube", random=False)

    d = np.load("output/BendTube/0000.npy", allow_pickle=True).item()
    
    # x, v, F, C, p1, p2, p3 = state['state']
    states_xvfcp = state['state']
    n_grips = d['shape_states'].shape[2]
    n_frames = d['shape_states'].shape[1]

    shape_states_ = d['shape_states'][0][0]
    
    for i_grip in range(n_grips):
        states_xvfcp[4+i_grip][:3] = shape_states_[i_grip][0:3]
        states_xvfcp[4+i_grip][3:] = shape_states_[i_grip][6:10]

#     p1[:3] = shape_states_[0][0:3]
#     p1[3:] = shape_states_[0][6:10]
#     p2[:3] = shape_states_[1][0:3]
#     p2[3:] = shape_states_[1][6:10]
#     p3[:3] = shape_states_[2][0:3]
#     p3[3:] = shape_states_[2][6:10]


    new_state = {
        'state': states_xvfcp,
        'is_copy': state['is_copy'],
        'softness': state['softness'],
    }
    env.set_state(**new_state)


    positions = []
    shape_states = []


    images = []
    states = []

    for frame in range(n_frames-1):
        print(f'{frame}', end="\r",)

        state = env.get_state()
        
        positions.append(np.concatenate((state['state'][0], np.ones([len(state['state'][0]), 1])), 1))
        sts = np.array(state['state'][4:])
        shape_states.append(np.concatenate([sts[:, :3], sts[:, :3], sts[:, 3:], sts[:, 3:]], axis=1))
        

        # env.step(calc_shape_states_dx(frame * dt, gripper_config))
        env.step((d['shape_states'][0][frame+1][:, 0:3] - d['shape_states'][0][frame][:, 0:3]).flatten() * 100)

        images.append(env.render('rgb_array'))

    state = env.get_state()

    positions.append(np.concatenate((state['state'][0], np.ones([len(state['state'][0]), 1])), 1))

    sts = np.array(state['state'][4:])
    shape_states.append(np.concatenate([sts[:, :3], sts[:, :3], sts[:, 3:], sts[:, 3:]], axis=1))

    states = {
            'positions': np.array([positions]),
            'shape_states': np.array([shape_states]),
            'E': params[param_i][0], 
            'YS': params[param_i][1], 
            'nu': params[param_i][2],
            'scene_info': d['scene_info']
            }

    with open('params/{:0>4}.npy'.format(str(param_i)), 'wb') as f:
            np.save(f, states)


    animate(images, 'params/{:0>4}.webm'.format(str(param_i)))

animating params/0000.webm
Moviepy - Building video params/0000.webm.
Moviepy - Writing video params/0000.webm



Moviepy - Done !
Moviepy - video ready params/0000.webm
animating params/0001.webm
Moviepy - Building video params/0001.webm.
Moviepy - Writing video params/0001.webm



Moviepy - Done !
Moviepy - video ready params/0001.webm
animating params/0002.webm
Moviepy - Building video params/0002.webm.
Moviepy - Writing video params/0002.webm



Moviepy - Done !
Moviepy - video ready params/0002.webm


KeyboardInterrupt: 